In [ ]:
"""
ESM2 Protein Embeddings (Filtered)

Purpose:
- Generate one fixed-size protein language model embedding per gene_symbol using ESM2.
- Filter sequences by length to avoid OOM and keep GPU-safe runs.

What this notebook does:
- Loads curated protein_coding_genes.csv, cleans sequences, computes seq_len.
- Splits into safe (≤ MAX_SAFE_LEN aa) and too-long sets; writes both lists.
- Runs ESM2 (MODEL_NAME, TARGET_LAYER) and mean-pools per-sequence token reps.
- Saves:
  - Torch file with {embeddings [genes×dim], genes [order], model name}
  - CSV gene order for downstream alignment.

Reasons:
- These embeddings are the gene-identity vectors consumed by the model.
- Dimensionality and order must match training/inference configs exactly.
- Reproducible, filtered set prevents memory issues and ensures consistency.

Inputs:  ./data/ensembl/protein_coding_genes.csv
Outputs: ./data/embeddings/{MODEL_NAME}_gene_embeddings.pt, gene_order.csv
"""

import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from esm import pretrained
from tqdm import tqdm

# ============================================================
# CONFIG
# ============================================================
PROTEIN_CSV = "./data/ensembl/protein_coding_genes.csv"
MODEL_NAME = "esm2_t6_8M_UR50D"
TARGET_LAYER = 6
MAX_SAFE_LEN = 3000
OUTDIR = "./data/embeddings"
FILTER_DIR = "./data/ensembl/filtered"

os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(FILTER_DIR, exist_ok=True)

BATCH_SIZE = 1  # conservative for GPU
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ============================================================
# LOAD DATA
# ============================================================
df = pd.read_csv(PROTEIN_CSV)
df["seq"] = df["seq"].astype(str).str.upper().str.replace("*", "", regex=False)
df["seq_len"] = df["seq"].str.len()

safe_df = df[df["seq_len"] <= MAX_SAFE_LEN].reset_index(drop=True)
long_df = df[df["seq_len"] > MAX_SAFE_LEN].reset_index(drop=True)

print(f"✅ Loaded {len(df):,} total protein-coding genes")
print(f"✅ {len(safe_df):,} sequences ≤ {MAX_SAFE_LEN} aa (GPU safe)")
print(f"⚠️ {len(long_df):,} sequences > {MAX_SAFE_LEN} aa (skipped)")

safe_path = os.path.join(FILTER_DIR, "safe_sequences.csv")
long_path = os.path.join(FILTER_DIR, "too_long_sequences.csv")

safe_df[["gene_symbol", "seq_len"]].to_csv(safe_path, index=False)
long_df[["gene_symbol", "seq_len"]].to_csv(long_path, index=False)
print(f"💾 Saved safe/long sequence metadata to {FILTER_DIR}")

# ============================================================
# LOAD MODEL
# ============================================================
print(f"\n🔧 Loading model: {MODEL_NAME}")
model, alphabet = pretrained.__dict__[MODEL_NAME]()
if torch.cuda.device_count() > 1:
    print(f"🧠 Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)
model = model.to(DEVICE).eval()
batch_converter = alphabet.get_batch_converter()
print("✅ Model ready")

# ============================================================
# DATASET + DATALOADER
# ============================================================
class ProteinDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return row["seq"], row["gene_symbol"]

def collate_batch(batch):
    seqs, ids = zip(*batch)
    batch_labels, batch_strs, batch_tokens = batch_converter(list(zip(ids, seqs)))
    lengths = (batch_tokens != alphabet.padding_idx).sum(1)
    return batch_tokens.to(DEVICE), lengths, ids

dataset = ProteinDataset(safe_df)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

# ============================================================
# GENERATE EMBEDDINGS
# ============================================================
embeddings, gene_order = [], []
print("\n🚀 Generating embeddings...")

with torch.no_grad():
    for batch_tokens, batch_lens, ids in tqdm(data_loader, total=len(data_loader)):
        # Forward with correct layer exposure under DP or single-GPU
        if isinstance(model, torch.nn.DataParallel):
            results = model.module(batch_tokens, repr_layers=[TARGET_LAYER], return_contacts=False)
        else:
            results = model(batch_tokens, repr_layers=[TARGET_LAYER], return_contacts=False)

        reps = results["representations"][TARGET_LAYER]

        for i, tokens_len in enumerate(batch_lens):
            # Mean-pool over residue tokens (exclude special tokens)
            seq_rep = reps[i, 1:tokens_len - 1].mean(0).cpu()
            embeddings.append(seq_rep)
            gene_order.append(ids[i])

# ============================================================
# SAVE RESULTS
# ============================================================
E_ESM2 = torch.stack(embeddings)
save_pt = os.path.join(OUTDIR, f"{MODEL_NAME}_gene_embeddings.pt")
save_csv = os.path.join(OUTDIR, f"{MODEL_NAME}_gene_order.csv")

torch.save({"embeddings": E_ESM2, "genes": gene_order, "model": MODEL_NAME}, save_pt)
pd.Series(gene_order, name="gene_symbol").to_csv(save_csv, index=False)

print(f"\n✅ Saved embeddings → {save_pt}")
print(f"✅ Shape: {E_ESM2.shape} [genes × dim={E_ESM2.shape[1]}]")


✅ Loaded 19,477 total protein-coding genes
✅ 19,357 sequences ≤ 3000 aa (GPU safe)
⚠️ 120 sequences > 3000 aa (skipped)
💾 Saved safe/long sequence metadata to ./data/ensembl/filtered

🔧 Loading model: esm2_t6_8M_UR50D
🧠 Using 2 GPUs
✅ Model ready

🚀 Generating embeddings...


100%|██████████| 19357/19357 [04:58<00:00, 64.84it/s]



✅ Saved embeddings → ./data/embeddings/esm2_t6_8M_UR50D_gene_embeddings.pt
✅ Shape: torch.Size([19357, 320]) [genes × dim=320]


In [13]:
"""
generate_esm2_embeddings.py
Generates one ESM2 embedding per gene_symbol.
No truncation or assumptions — uses ESM defaults.
"""

import os
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from esm import pretrained
from tqdm import tqdm
from Bio import SeqIO

# ============================================================
# CONFIG
# ============================================================
FASTA_PATH = "./data/ensembl/Homo_sapiens.GRCh38.pep.all.fa"
# MODEL_NAME = "esm2_t33_650M_UR50D"
# MODEL_NAME = "esm2_t12_35M_UR50D"
MODEL_NAME = "esm2_t6_8M_UR50D"
target_layer = 6
OUTDIR = "./data/embeddings"
os.makedirs(OUTDIR, exist_ok=True)

BATCH_SIZE = 1  # adjust for GPU memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# ============================================================
# LOAD MODEL
# ============================================================
print(f"🔧 Loading model: {MODEL_NAME}")
model, alphabet = pretrained.__dict__[MODEL_NAME]()
if torch.cuda.device_count() > 1:
    print(f"🧠 Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)
model = model.to(DEVICE).eval()
print("✅ Model ready")

batch_converter = alphabet.get_batch_converter()

# ============================================================
# PARSE FASTA
# ============================================================
records = []
for rec in SeqIO.parse(FASTA_PATH, "fasta"):
    parts = {k: v for k, v in (tok.split(":", 1) for tok in rec.description.split() if ":" in tok)}
    gene_symbol = parts.get("gene_symbol")
    if not gene_symbol or not gene_symbol.strip():
        continue

    seq = str(rec.seq).strip().upper().replace("*", "")  # remove stop codons
    records.append({"gene_symbol": gene_symbol.strip(), "seq": seq})

df = pd.DataFrame(records).drop_duplicates("gene_symbol").reset_index(drop=True)
print(f"✅ Loaded {len(df):,} unique protein-coding genes")

# ============================================================
# DATASET + DATALOADER
# ============================================================
class ProteinDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return row["seq"], row["gene_symbol"]

def collate_batch(batch):
    seqs, ids = zip(*batch)
    batch_labels, batch_strs, batch_tokens = batch_converter(list(zip(ids, seqs)))
    lengths = (batch_tokens != alphabet.padding_idx).sum(1)
    return batch_tokens.to(DEVICE), lengths, ids

dataset = ProteinDataset(safe_df)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

# ============================================================
# GENERATE EMBEDDINGS
# ============================================================
embeddings, gene_order = [], []
print("🚀 Generating embeddings...")

with torch.no_grad():
    for batch_tokens, batch_lens, ids in tqdm(data_loader, total=len(data_loader)):
        # results = model(batch_tokens, repr_layers=[33], return_contacts=False)
        target_layer = 6  # for esm2_t12_35M_UR50D
        results = model.module(batch_tokens, repr_layers=[target_layer], return_contacts=False)
        reps = results["representations"][target_layer]

        # results = model.module(batch_tokens, repr_layers=[33], return_contacts=False)
        # reps = results["representations"][33]

        for i, tokens_len in enumerate(batch_lens):
            seq_rep = reps[i, 1:tokens_len - 1].mean(0).cpu()  # mean-pool actual length
            embeddings.append(seq_rep)
            gene_order.append(ids[i])

# ============================================================
# SAVE RESULTS
# ============================================================
E_ESM2 = torch.stack(embeddings)
save_path = os.path.join(OUTDIR, f"{MODEL_NAME}_gene_embeddings.pt")

torch.save(
    {"embeddings": E_ESM2, "genes": gene_order, "model": MODEL_NAME},
    save_path
)

pd.Series(gene_order, name="gene_symbol").to_csv(
    os.path.join(OUTDIR, f"{MODEL_NAME}_gene_order.csv"), index=False
)

print(f"\n✅ Saved embeddings → {save_path}")
print(f"✅ Shape: {E_ESM2.shape} [genes × dim={E_ESM2.shape[1]}]")


🔧 Loading model: esm2_t6_8M_UR50D
🧠 Using 2 GPUs
✅ Model ready
✅ Loaded 19,885 unique protein-coding genes
🚀 Generating embeddings...


100%|██████████| 19765/19765 [05:45<00:00, 57.18it/s]



✅ Saved embeddings → ./data/embeddings/esm2_t6_8M_UR50D_gene_embeddings.pt
✅ Shape: torch.Size([19765, 320]) [genes × dim=320]


In [14]:
import pandas as pd
import torch

# Preview filtered lists
safe = pd.read_csv("./data/ensembl/filtered/safe_sequences.csv")
long = pd.read_csv("./data/ensembl/filtered/too_long_sequences.csv")

print("=== Safe sequences (short) ===")
print(safe.head(), "\nTotal:", len(safe))

print("\n=== Too long sequences (>3000 aa) ===")
print(long.head(), "\nTotal:", len(long))

# Preview embeddings
emb = torch.load("./data/embeddings/esm2_t6_8M_UR50D_gene_embeddings.pt")
E = emb["embeddings"]
genes = emb["genes"]

print("\n=== Embeddings preview ===")
print(f"Tensor shape: {E.shape}  →  ({E.shape[0]} genes × {E.shape[1]} dims)")
print("First 5 genes:", genes[:5])
print("First embedding vector (first 10 dims):", E[0][:10])


=== Safe sequences (short) ===
   gene_symbol  seq_len
0  IGHV1OR21-1      117
1      IGKV5-2      115
2      IGKV1-5      117
3      IGKV1-6      117
4      IGKV3-7      116 
Total: 19765

=== Too long sequences (>3000 aa) ===
  gene_symbol  seq_len
0      MUC5AC     5654
1        MUC2     5130
2       FCGBP     5405
3        APOB     4563
4       MUC16    15287 
Total: 120

=== Embeddings preview ===
Tensor shape: torch.Size([19765, 320])  →  (19765 genes × 320 dims)
First 5 genes: ['IGHV1OR21-1', 'IGKV5-2', 'IGKV1-5', 'IGKV1-6', 'IGKV3-7']
First embedding vector (first 10 dims): tensor([-0.2124, -0.1109,  0.0512, -0.0074,  0.3155, -0.1443,  0.0293, -0.0276,
        -0.2905,  0.0407])


/tmp/ipykernel_80764/1784561354.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load("./data/embeddings/esm2_t6_8M_UR50D_gene_embeddings.pt")
